In [ ]:
# Import necessary libraries
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow import keras
from tensorflow.keras import layers
import pandas as pd

# Load the dataset (replace 'path/to/your-dataset.csv' with your actual file path)
data = pd.read_csv('/content/csv_result-ECG5000_TEST.csv')  # Replace with your dataset path

# Convert dataset to a NumPy array
X = data.values
X = X.astype('float32') / 255.0  # Normalize if needed

# Stage b: Split the dataset
X_train, X_test = train_test_split(X, test_size=0.2, random_state=42)

# Stage c: Scale the data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Stage c: Define the Encoder and Decoder
input_dim = X_train.shape[1]
encoding_dim = 32
input_layer = layers.Input(shape=(input_dim,))
encoder = layers.Dense(encoding_dim, activation='relu')(input_layer)
decoder = layers.Dense(input_dim, activation='sigmoid')(encoder)
autoencoder = keras.Model(inputs=input_layer, outputs=decoder)

# Stage d: Compile the model
autoencoder.compile(optimizer='adam', loss='mean_squared_error')

# Stage e: Train the autoencoder
history = autoencoder.fit(X_train, X_train, epochs=35, batch_size=256, validation_data=(X_test, X_test), shuffle=True)

# Predict and calculate reconstruction error for anomaly detection
X_test_pred = autoencoder.predict(X_test)
reconstruction_error = np.mean(np.square(X_test - X_test_pred), axis=1)

# Define threshold and identify anomalies
threshold = np.percentile(reconstruction_error, 95)
anomalies = reconstruction_error > threshold
print(f"Number of anomalies detected: {np.sum(anomalies)}")

# Plot training and validation loss
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Model Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()